In [17]:
from pathlib import Path
import sys
sys.path.append('../')
from quantized_fast_forward.fast_forward.index import H5Index, Mode
from quantized_fast_forward.fast_forward.encoder import QueryEncoder
from quantized_fast_forward.fast_forward.ranking import Ranking
from pyserini.search.faiss import AggretrieverQueryEncoder
import numpy as np

In [18]:
# define the query encoders
class FFAggretrieverQueryEncoder(QueryEncoder):
    def __init__(self, model_name, device="cpu"):
        self._enc = AggretrieverQueryEncoder(model_name, device=device)

    # for whatever reason, pyserini flattens the outputs, so we need this
    # TODO: implement batch encoding
    def encode(self, queries):
        return np.array([self._enc.encode(q) for q in queries])

In [19]:
# define the index
index = H5Index.from_disk(
    Path("/home/catalinlup/MyWorkspace/MasterThesis/datasets/h5_indices/faiss.msmarco-v1-passage.aggretriever-cocondenser.h5"),
    FFAggretrieverQueryEncoder("castorini/aggretriever-cocondenser"),
)

In [20]:
from datasets import get_dataset

CHOSEN_DATASET = 'dev'

queries, q_rels = get_dataset(CHOSEN_DATASET)

queries_psg20 = {x.query_id: x.text for x in queries}

/home/catalinlup/MyWorkspace/MasterThesis/ExperimentsCodebase/experiment_notebooks/datasets.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  queries_list.append(GenericQuery(queries.iloc[i][0], queries.iloc[i][1]))
/home/catalinlup/MyWorkspace/MasterThesis/ExperimentsCodebase/experiment_notebooks/datasets.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  qrels_list.append(TrecQrel(qrels.iloc[i][0], qrels.iloc[i][2], qrels.iloc[i][3], qrels.iloc[i][1]))


In [21]:
# load the ranking
splade_ranking = Ranking.from_file(Path("/home/catalinlup/MyWorkspace/MasterThesis/datasets/run_files/ms_marco_psg_splade_ranking.tsv"))
print(f"loaded run with {len(splade_ranking)} queries")

# print(splade_ranking)

for q_id in splade_ranking.q_ids:
    assert q_id in queries_psg20

loaded run with 200 queries


AssertionError: 

In [ ]:
# run the re-ranking experiment
alpha = 0.0
result = index.get_scores(
    splade_ranking,
    queries_psg20,
    alpha=alpha,
)

In [ ]:
from ir_measures import calc_aggregate, nDCG, AP, RR

# print(result)

METRICS = [nDCG@10, AP(rel=2)@1000, RR(rel=2)@10]

print(
    "SPLADE",
    calc_aggregate(METRICS, q_rels, splade_ranking.run),
)
print(
    f"SPLADE, Aggretriever (alpha={alpha})",
    calc_aggregate(METRICS, q_rels, result[alpha].run),
)